### Installing Dependencies

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\renato.valentim\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


### Imports & Paths

In [2]:
from utils import *

In [3]:

CORPUS_PATH = f"{MAIN_PATH}/ofertas_es_07032025.csv"

### Corpus Data

In [6]:
df_corpus = (
  spark.read
  .option("header", True)
  .option("inferSchema", True)
  .option("multiline", True)
  .option("quote", '"')
  .option("escape", "\\")
  .option("sep", ";")
  .option("ignoreLeadingWhiteSpace", True)
  .option("ignoreTrailingWhiteSpace", True)
  .csv(CORPUS_PATH)
  .withColumnRenamed("DESC_PRODUCT", "desc_product")
  .select('id_product', 'desc_product')
)

In [7]:
terms = ["OUTLET", "ANUNCIO", "OLD"]
df_corpus = remove_terms_from_column(df_corpus, terms, column_name="desc_product")

In [8]:
print("Número de Ofertas:", f"{df_corpus.count():,}")

Número de Ofertas: 897,211


In [9]:
df_corpus.show(truncate=False)

+----------+---------------------------------------------------------------------------------+
|id_product|desc_product                                                                     |
+----------+---------------------------------------------------------------------------------+
|77972762  |MESA DE ESCRITORIO KUANDRA NOGAL E PRETA                                         |
|113606819 |RACK PARA TV ATE 43 POLEGADAS CARINHO 1 PT CINAMOMO                              |
|127468252 |COLCHAO ORTOPEDICO PREMIUM QUEEN                                                 |
|113440274 |BALCAO PARA COZINHA 2 PORTAS 3 GAVETAS PROVENCAIS SEM TAMPO ILUMINATA            |
|82899687  |MESA AMSTERDA TAMPO GIRATORIO DOBUE CEDRO OFF WHITE 136M                         |
|104642417 |COLCHAO SOLTEIRO ORTOBOM LIGHT ORTOPEDICO 88X188X24                              |
|124330736 |COLCHAO CASAL ANJOS MOLAS SUPERLASTIC CLASSIC 1                                  |
|93263819  |PAINEL LED SOB QUAD CCT SMART 18W 1300

### Elastic Search

In [10]:
# Coletar as linhas do DataFrame e preparar para envio em lote
rows = df_corpus.collect()
actions = list(generate_actions(rows))

In [11]:
print("Aqui estão alguns exemplos das actions criadas para index no ES"), actions[:2]

Aqui estão alguns exemplos das actions criadas para index no ES


(None,
 [{'_index': 'ofertas_corpus',
   '_id': '77972762',
   '_source': {'desc_product': 'MESA DE ESCRITORIO KUANDRA NOGAL E PRETA'}},
  {'_index': 'ofertas_corpus',
   '_id': '113606819',
   '_source': {'desc_product': 'RACK PARA TV ATE 43 POLEGADAS CARINHO 1 PT CINAMOMO'}}])

In [12]:
# Enviar os dados para o Elasticsearch
bulk(es, actions)

print("Dados do corpus carregados com sucesso!")

Dados do corpus carregados com sucesso!
